# 준비
라이브러리, 파라미터 세팅

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import re

In [2]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/99_Ensemble_ML/'

TEST_SIZE = 0.2
RANDOM_SEED = 42

# 데이터 불러오기

In [3]:
# 테스트(캐글) 데이터 로드
df_ensemble = pd.read_csv(DATA_IN_PATH + 'ko_data.csv', encoding = 'cp949')
df_ensemble.columns = ['id','document']  # 전처리 일괄 수행을 위해 컬럼명 변경(학습 데이터셋과 동일하게)
df_ensemble.head()

,id,document
0,0,정말 많이 울었던 영화입니다.
1,1,시간 낭비예요.
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.


In [4]:
# 앙상블 대상 파일 확인
fnames = glob.glob(DATA_OUT_PATH+'*.csv')
fnames.sort()

for i in fnames:
    print(i.split('/')[-1])

NSMC_LightGBM_076810.csv
NSMC_LogisticRegression_083622.csv
NSMC_RandomForest_081745.csv
NSMC_XGBoost_076238.csv


In [5]:
len(fnames)

4

In [6]:
for i in range(len(fnames)):
    tmp = pd.read_csv(fnames[i])
    tmp.drop('Id', axis=1, inplace=True)
    df_ensemble = pd.concat([df_ensemble, tmp], axis=1)

In [7]:
df_ensemble

,id,document,Predicted,Predicted,Predicted,Predicted
0,0,정말 많이 울었던 영화입니다.,1,1,1,1
1,1,시간 낭비예요.,0,0,0,0
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.,0,0,1,0
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!,1,1,1,1
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.,0,0,0,0
...,...,...,...,...,...,...
11182,11182,이 영화를 커플에게 추천합니다. 영화관에 가다보면 평생 잊지 못할 추억이 하나 생길...,1,1,1,1
11183,11183,심심__ 그냥 한효주 cf,0,0,0,0
11184,11184,공감해서 눈물나는 영화. 안 보신분들이 전부 제가 울었다고 하면 의아해하실텐데 보면...,0,0,0,0
11185,11185,오토바이 신은 최고네요.,1,1,1,1


# 모델 앙상블(Majority Voting) 방식

In [8]:
len(df_ensemble)

11187

In [9]:
len(fnames)+1

5

In [16]:
ensemble_result = []

for i in range(len(df_ensemble)):  # Id 별
    cnt_pos = 0  # 긍정 카운트
    cnt_neg = 0  # 부정 카운트
    
    for j in range(2,len(fnames)+2):  # 모델 별
        if df_ensemble.iloc[i,j] == 1:
            cnt_pos += 1
        if df_ensemble.iloc[i,j] == 0:
            cnt_neg += 1

    print(i, 'positive_sentiment:', cnt_pos \
          ,  'negative_sentiment:', cnt_neg )
    
#     # 4개 대상일 때 사용
#     ensemble_result.append(int(cnt_pos/(len(fnames)/2+1)))  # 4개 대상일 때 3으로 나눈다.

    # 4개 대상일 때 사용
    if cnt_pos >= 3:
        ensemble_result.append(1)
    else:
        ensemble_result.append(0)

#     # 5개 대상일 때 사용
#     if cnt_pos >= 3:
#         ensemble_result.append(1)
#     else:
#         ensemble_result.append(0)
    
df_ensemble["Ensemble_Result"] = ensemble_result

0 positive_sentiment: 4 negative_sentiment: 0
1 positive_sentiment: 0 negative_sentiment: 4
2 positive_sentiment: 1 negative_sentiment: 3
3 positive_sentiment: 4 negative_sentiment: 0
4 positive_sentiment: 0 negative_sentiment: 4
5 positive_sentiment: 4 negative_sentiment: 0
6 positive_sentiment: 3 negative_sentiment: 1
7 positive_sentiment: 0 negative_sentiment: 4
8 positive_sentiment: 0 negative_sentiment: 4
9 positive_sentiment: 2 negative_sentiment: 2
10 positive_sentiment: 4 negative_sentiment: 0
11 positive_sentiment: 0 negative_sentiment: 4
12 positive_sentiment: 0 negative_sentiment: 4
13 positive_sentiment: 0 negative_sentiment: 4
14 positive_sentiment: 0 negative_sentiment: 4
15 positive_sentiment: 4 negative_sentiment: 0
16 positive_sentiment: 4 negative_sentiment: 0
17 positive_sentiment: 4 negative_sentiment: 0
18 positive_sentiment: 4 negative_sentiment: 0
19 positive_sentiment: 4 negative_sentiment: 0
20 positive_sentiment: 4 negative_sentiment: 0
21 positive_sentiment: 

367 positive_sentiment: 3 negative_sentiment: 1
368 positive_sentiment: 2 negative_sentiment: 2
369 positive_sentiment: 0 negative_sentiment: 4
370 positive_sentiment: 0 negative_sentiment: 4
371 positive_sentiment: 1 negative_sentiment: 3
372 positive_sentiment: 2 negative_sentiment: 2
373 positive_sentiment: 0 negative_sentiment: 4
374 positive_sentiment: 0 negative_sentiment: 4
375 positive_sentiment: 1 negative_sentiment: 3
376 positive_sentiment: 4 negative_sentiment: 0
377 positive_sentiment: 4 negative_sentiment: 0
378 positive_sentiment: 0 negative_sentiment: 4
379 positive_sentiment: 2 negative_sentiment: 2
380 positive_sentiment: 4 negative_sentiment: 0
381 positive_sentiment: 0 negative_sentiment: 4
382 positive_sentiment: 4 negative_sentiment: 0
383 positive_sentiment: 0 negative_sentiment: 4
384 positive_sentiment: 3 negative_sentiment: 1
385 positive_sentiment: 4 negative_sentiment: 0
386 positive_sentiment: 0 negative_sentiment: 4
387 positive_sentiment: 4 negative_senti

741 positive_sentiment: 2 negative_sentiment: 2
742 positive_sentiment: 4 negative_sentiment: 0
743 positive_sentiment: 2 negative_sentiment: 2
744 positive_sentiment: 0 negative_sentiment: 4
745 positive_sentiment: 4 negative_sentiment: 0
746 positive_sentiment: 4 negative_sentiment: 0
747 positive_sentiment: 4 negative_sentiment: 0
748 positive_sentiment: 4 negative_sentiment: 0
749 positive_sentiment: 2 negative_sentiment: 2
750 positive_sentiment: 0 negative_sentiment: 4
751 positive_sentiment: 0 negative_sentiment: 4
752 positive_sentiment: 4 negative_sentiment: 0
753 positive_sentiment: 0 negative_sentiment: 4
754 positive_sentiment: 4 negative_sentiment: 0
755 positive_sentiment: 0 negative_sentiment: 4
756 positive_sentiment: 0 negative_sentiment: 4
757 positive_sentiment: 0 negative_sentiment: 4
758 positive_sentiment: 2 negative_sentiment: 2
759 positive_sentiment: 0 negative_sentiment: 4
760 positive_sentiment: 1 negative_sentiment: 3
761 positive_sentiment: 4 negative_senti

1119 positive_sentiment: 0 negative_sentiment: 4
1120 positive_sentiment: 0 negative_sentiment: 4
1121 positive_sentiment: 3 negative_sentiment: 1
1122 positive_sentiment: 0 negative_sentiment: 4
1123 positive_sentiment: 0 negative_sentiment: 4
1124 positive_sentiment: 4 negative_sentiment: 0
1125 positive_sentiment: 0 negative_sentiment: 4
1126 positive_sentiment: 0 negative_sentiment: 4
1127 positive_sentiment: 1 negative_sentiment: 3
1128 positive_sentiment: 2 negative_sentiment: 2
1129 positive_sentiment: 0 negative_sentiment: 4
1130 positive_sentiment: 1 negative_sentiment: 3
1131 positive_sentiment: 3 negative_sentiment: 1
1132 positive_sentiment: 4 negative_sentiment: 0
1133 positive_sentiment: 2 negative_sentiment: 2
1134 positive_sentiment: 4 negative_sentiment: 0
1135 positive_sentiment: 4 negative_sentiment: 0
1136 positive_sentiment: 3 negative_sentiment: 1
1137 positive_sentiment: 0 negative_sentiment: 4
1138 positive_sentiment: 0 negative_sentiment: 4
1139 positive_sentim

1501 positive_sentiment: 4 negative_sentiment: 0
1502 positive_sentiment: 4 negative_sentiment: 0
1503 positive_sentiment: 0 negative_sentiment: 4
1504 positive_sentiment: 0 negative_sentiment: 4
1505 positive_sentiment: 3 negative_sentiment: 1
1506 positive_sentiment: 4 negative_sentiment: 0
1507 positive_sentiment: 0 negative_sentiment: 4
1508 positive_sentiment: 1 negative_sentiment: 3
1509 positive_sentiment: 0 negative_sentiment: 4
1510 positive_sentiment: 0 negative_sentiment: 4
1511 positive_sentiment: 0 negative_sentiment: 4
1512 positive_sentiment: 4 negative_sentiment: 0
1513 positive_sentiment: 4 negative_sentiment: 0
1514 positive_sentiment: 0 negative_sentiment: 4
1515 positive_sentiment: 4 negative_sentiment: 0
1516 positive_sentiment: 4 negative_sentiment: 0
1517 positive_sentiment: 1 negative_sentiment: 3
1518 positive_sentiment: 0 negative_sentiment: 4
1519 positive_sentiment: 0 negative_sentiment: 4
1520 positive_sentiment: 0 negative_sentiment: 4
1521 positive_sentim

...

9419 positive_sentiment: 0 negative_sentiment: 4
9420 positive_sentiment: 0 negative_sentiment: 4
9421 positive_sentiment: 4 negative_sentiment: 0
9422 positive_sentiment: 4 negative_sentiment: 0
9423 positive_sentiment: 2 negative_sentiment: 2
9424 positive_sentiment: 0 negative_sentiment: 4
9425 positive_sentiment: 0 negative_sentiment: 4
9426 positive_sentiment: 0 negative_sentiment: 4
9427 positive_sentiment: 2 negative_sentiment: 2
9428 positive_sentiment: 4 negative_sentiment: 0
9429 positive_sentiment: 4 negative_sentiment: 0
9430 positive_sentiment: 0 negative_sentiment: 4
9431 positive_sentiment: 4 negative_sentiment: 0
9432 positive_sentiment: 4 negative_sentiment: 0
9433 positive_sentiment: 1 negative_sentiment: 3
9434 positive_sentiment: 4 negative_sentiment: 0
9435 positive_sentiment: 4 negative_sentiment: 0
9436 positive_sentiment: 2 negative_sentiment: 2
9437 positive_sentiment: 0 negative_sentiment: 4
9438 positive_sentiment: 0 negative_sentiment: 4
9439 positive_sentim

9789 positive_sentiment: 4 negative_sentiment: 0
9790 positive_sentiment: 4 negative_sentiment: 0
9791 positive_sentiment: 2 negative_sentiment: 2
9792 positive_sentiment: 4 negative_sentiment: 0
9793 positive_sentiment: 0 negative_sentiment: 4
9794 positive_sentiment: 0 negative_sentiment: 4
9795 positive_sentiment: 3 negative_sentiment: 1
9796 positive_sentiment: 4 negative_sentiment: 0
9797 positive_sentiment: 4 negative_sentiment: 0
9798 positive_sentiment: 0 negative_sentiment: 4
9799 positive_sentiment: 0 negative_sentiment: 4
9800 positive_sentiment: 4 negative_sentiment: 0
9801 positive_sentiment: 0 negative_sentiment: 4
9802 positive_sentiment: 4 negative_sentiment: 0
9803 positive_sentiment: 4 negative_sentiment: 0
9804 positive_sentiment: 3 negative_sentiment: 1
9805 positive_sentiment: 0 negative_sentiment: 4
9806 positive_sentiment: 0 negative_sentiment: 4
9807 positive_sentiment: 0 negative_sentiment: 4
9808 positive_sentiment: 0 negative_sentiment: 4
9809 positive_sentim

10163 positive_sentiment: 4 negative_sentiment: 0
10164 positive_sentiment: 4 negative_sentiment: 0
10165 positive_sentiment: 4 negative_sentiment: 0
10166 positive_sentiment: 4 negative_sentiment: 0
10167 positive_sentiment: 0 negative_sentiment: 4
10168 positive_sentiment: 1 negative_sentiment: 3
10169 positive_sentiment: 0 negative_sentiment: 4
10170 positive_sentiment: 0 negative_sentiment: 4
10171 positive_sentiment: 4 negative_sentiment: 0
10172 positive_sentiment: 2 negative_sentiment: 2
10173 positive_sentiment: 4 negative_sentiment: 0
10174 positive_sentiment: 0 negative_sentiment: 4
10175 positive_sentiment: 1 negative_sentiment: 3
10176 positive_sentiment: 0 negative_sentiment: 4
10177 positive_sentiment: 4 negative_sentiment: 0
10178 positive_sentiment: 0 negative_sentiment: 4
10179 positive_sentiment: 3 negative_sentiment: 1
10180 positive_sentiment: 0 negative_sentiment: 4
10181 positive_sentiment: 4 negative_sentiment: 0
10182 positive_sentiment: 4 negative_sentiment: 0


10551 positive_sentiment: 3 negative_sentiment: 1
10552 positive_sentiment: 0 negative_sentiment: 4
10553 positive_sentiment: 1 negative_sentiment: 3
10554 positive_sentiment: 4 negative_sentiment: 0
10555 positive_sentiment: 4 negative_sentiment: 0
10556 positive_sentiment: 1 negative_sentiment: 3
10557 positive_sentiment: 4 negative_sentiment: 0
10558 positive_sentiment: 4 negative_sentiment: 0
10559 positive_sentiment: 0 negative_sentiment: 4
10560 positive_sentiment: 2 negative_sentiment: 2
10561 positive_sentiment: 0 negative_sentiment: 4
10562 positive_sentiment: 4 negative_sentiment: 0
10563 positive_sentiment: 4 negative_sentiment: 0
10564 positive_sentiment: 4 negative_sentiment: 0
10565 positive_sentiment: 0 negative_sentiment: 4
10566 positive_sentiment: 0 negative_sentiment: 4
10567 positive_sentiment: 0 negative_sentiment: 4
10568 positive_sentiment: 4 negative_sentiment: 0
10569 positive_sentiment: 1 negative_sentiment: 3
10570 positive_sentiment: 0 negative_sentiment: 4


10933 positive_sentiment: 0 negative_sentiment: 4
10934 positive_sentiment: 3 negative_sentiment: 1
10935 positive_sentiment: 4 negative_sentiment: 0
10936 positive_sentiment: 0 negative_sentiment: 4
10937 positive_sentiment: 4 negative_sentiment: 0
10938 positive_sentiment: 0 negative_sentiment: 4
10939 positive_sentiment: 0 negative_sentiment: 4
10940 positive_sentiment: 3 negative_sentiment: 1
10941 positive_sentiment: 0 negative_sentiment: 4
10942 positive_sentiment: 4 negative_sentiment: 0
10943 positive_sentiment: 1 negative_sentiment: 3
10944 positive_sentiment: 0 negative_sentiment: 4
10945 positive_sentiment: 3 negative_sentiment: 1
10946 positive_sentiment: 4 negative_sentiment: 0
10947 positive_sentiment: 4 negative_sentiment: 0
10948 positive_sentiment: 2 negative_sentiment: 2
10949 positive_sentiment: 2 negative_sentiment: 2
10950 positive_sentiment: 0 negative_sentiment: 4
10951 positive_sentiment: 4 negative_sentiment: 0
10952 positive_sentiment: 3 negative_sentiment: 1


In [17]:
df_ensemble.head(20)

,id,document,Predicted,Predicted,Predicted,Predicted,Ensemble_Result
0,0,정말 많이 울었던 영화입니다.,1,1,1,1,1
1,1,시간 낭비예요.,0,0,0,0,0
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.,0,0,1,0,0
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!,1,1,1,1,1
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.,0,0,0,0,0
5,5,잔잔한 감동을 주는 영화가 좋은 영화다. 줄리안 무어의 매력!,1,1,1,1,1
6,6,"프랑스 영화, 정말 재미없다는 말밖에...",1,1,1,0,1
7,7,이보다 더 자연스러울 수는 없다.,0,0,0,0,0
8,8,잠만 자고 있었는데~~,0,0,0,0,0
9,9,오프닝 씬이... (이 영화와 젊음에 대해 말한다.),0,1,1,0,0


In [18]:
df_ensemble.tail(20)

,id,document,Predicted,Predicted,Predicted,Predicted,Ensemble_Result
11167,11167,피아노 선율을 타고 영화를 즐겼습니다.,0,1,0,0,0
11168,11168,"""협녀, 최악의 기억"",",0,0,0,0,0
11169,11169,모야... 애니메이션 뒷얘기인 줄 알았으면 모야... 실망..,0,0,0,0,0
11170,11170,별로 기대하지 않고 영화를 봤는데 시간가는줄 모르고 재미있었어요. 과대 혹평이 많군...,1,1,1,1,1
11171,11171,어린 놈들이 담배만 피우는 영화다,0,0,0,0,0
11172,11172,엔딩이 너무 좋아 흔한 클리셰가 아니라 예측하기 어렵고 정말 좋았다.,1,1,1,1,1
11173,11173,"이런 훌륭한 작품에 1점 테러를 하다니, 무식한 놈들이 있나?",0,0,0,0,0
11174,11174,총을 쏠 때 너무 답답해서 그냥 잡는 게 아니라 액션이나 하려고 생각하는 부분이,0,0,0,0,0
11175,11175,너무 유치하더라. 알바 평점 조작 같아.,0,0,0,0,0
11176,11176,성룡액션도 좋고 성룡이가 부르는 노래를 핸드폰에 저장해서 듣고 있는데...9월의 아...,1,1,1,1,1


In [19]:
output = df_ensemble[['id','Ensemble_Result']].copy()
output = output.rename({'id':'Id','Ensemble_Result':'Predicted'}, axis='columns')
output.head()

,Id,Predicted
0,0,1
1,1,0
2,2,0
3,3,1
4,4,0


In [20]:
# 해당 경로가 없으면 생성
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

# csv파일 생성
output.to_csv("./data_out/NSMC_Ensemble_ML_NN.csv", index = False)  # 앙상블 결과, 캐글 제출 용도

### 캐글 제출 결과
**[2020.12.20]**<br>
NSMC_LightGBM_076810.csv<br>
NSMC_LogisticRegression_083622.csv<br>
NSMC_RandomForest_081745.csv<br>
NSMC_XGBoost_076238.csv<br>
조합(4개) => 0.78687